In [23]:
import gym
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import pandas as pd

# Load pre-trained BERT model and tokenizer


We're loading a pre-trained BERT (Bidirectional Encoder Representations from Transformers) model and its corresponding tokenizer using the Hugging Face Transformers library.

The code :
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
This line initializes a BERT tokenizer using the 'bert-base-uncased' pre-trained model.
The tokenizer is responsible for converting text into tokens, which can then be fed into the BERT model.

model = BertModel.from_pretrained('bert-base-uncased')
This line loads the pre-trained BERT model itself from the 'bert-base-uncased' checkpoint.

The model is a deep neural network that has been trained on a large corpus of text to understand contextual information in language.

After running these lines, we will have a BERT tokenizer (tokenizer) and a pre-trained BERT model (model) ready to use for natural language processing tasks.

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [25]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#Getting the database and dropping the spam column
df = pd.read_csv("/content/drive/MyDrive/email sec/Phishing_Email.csv").drop("Unnamed: 0", axis = 1)

# Creating database with Email Embeddings

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm import tqdm
import numpy as np

# Load your dataset
df = pd.read_csv("/content/drive/MyDrive/email sec/Phishing_Email.csv").drop("Unnamed: 0", axis = 1)
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

embeddings = []
types = []

# Process emails
for i in tqdm(range(len(df))):
    try:
        email_text = df["Email Text"].iloc[i]
    except KeyError:
        # This happens if there are no more values in the dataset
        break

    if pd.isna(email_text) or email_text == "empty":
        continue

    # Tokenize and encode email text
    try:
        inputs = tokenizer(email_text, return_tensors='pt', padding=True, truncation=True)
    except Exception as e:
        # Log the error and continue
        print(f"Error tokenizing email {i}: {e}")
        continue

    # Generate embeddings
    try:
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    except Exception as e:
        # Log the error and continue
        print(f"Error generating embeddings for email {i}: {e}")
        continue

    # Assign label
    if df["Email Type"].iloc[i] == "Safe Email":
        types.append(0)
    else:
        types.append(1)

    if i % 1000 == 0:
        print(i)

# Create DataFrame from embeddings and types
data = {'Email Embeddings': embeddings, 'Type': types}
df_updated = pd.DataFrame(data)

# Display the first few rows of the updated DataFrame
print(df_updated.head())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|          | 1/18650 [00:01<7:48:35,  1.51s/it]

0


  5%|▌         | 1001/18650 [26:11<3:24:45,  1.44it/s]

1000


  5%|▌         | 1019/18650 [26:29<7:26:48,  1.52s/it]

In [ ]:
# Save the DataFrame
df_updated.to_csv("/content/drive/MyDrive/email sec/email_data.csv", index=False)
# Optionally, you can use a more efficient format like HDF5 for large datasets
df_updated.to_hdf('email_data.h5', key='df', mode='w')


# Create the environment
n_states =
n_actions =
env = EmailSecurityEnv()
# Define the email security environment
 class EmailSecurityEnv(gym.Env):
   def __init__():
   def reset(self):
        Initialize email content as not spam
   def step(self, action):
         Simulate email content being checked
             Correct classification
         reward = 1
    else:
             Incorrect classification
        reward = -1



In [ ]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

class EmailSecurityEnv(gym.Env):
    def __init__(self, emails, labels):
        super(EmailSecurityEnv, self).__init__()

        # Store the emails and labels
        self.emails = emails
        self.labels = labels

        # Define action and observation space
        # Actions: 0 - not spam, 1 - spam
        self.action_space = spaces.Discrete(2)

        # Observations: email embeddings
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(emails.shape[1],), dtype=np.float32)

        # Initialize state
        self.current_index = 0

    def reset(self):
        # Reset the state to the first email
        self.current_index = 0
        return self.emails[self.current_index]

    def step(self, action):
        # Get the current email and its correct label
        email_embedding = self.emails[self.current_index]
        correct_label = self.labels[self.current_index]

        # Reward system
        reward = 1 if action == correct_label else -1

        # Move to the next email
        self.current_index += 1
        done = self.current_index >= len(self.emails)

        # Return step information
        return email_embedding, reward, done, {}

    def render(self, mode='human', close=False):
        # Optional: Implement if you want to visualize the environment
        pass

# Example usage
if __name__ == "__main__":
    # Load the DataFrame
    df_loaded = pd.read_csv('email_data.csv')

    # Extract embeddings and labels
    embeddings = np.array(df_loaded['Email Embeddings'].apply(lambda x: np.fromstring(x[1:-1], sep=' ')))
    types = np.array(df_loaded['Type'])

    # Create the environment
    env = EmailSecurityEnv(embeddings, types)

    # Example: Reset environment and take a random action
    obs = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()  # Replace with your agent's action
        obs, reward, done, info = env.step(action)
        print(f"Observation: {obs}, Action: {action}, Reward: {reward}, Done: {done}")


# After training,test email security classification




In [ ]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO  # Replace with your actual algorithm
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the custom environment
class EmailSecurityEnv(gym.Env):
    def __init__(self, emails, labels):
        super(EmailSecurityEnv, self).__init__()

        # Store the emails and labels
        self.emails = emails
        self.labels = labels

        # Define action and observation space
        self.action_space = spaces.Discrete(2)  # Actions: 0 - not spam, 1 - spam
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(emails.shape[1],), dtype=np.float32)

        # Initialize state
        self.current_index = 0

    def reset(self):
        # Reset the state to the first email
        self.current_index = 0
        return self.emails[self.current_index]

    def step(self, action):
        # Get the current email and its correct label
        email_embedding = self.emails[self.current_index]
        correct_label = self.labels[self.current_index]

        # Reward system
        reward = 1 if action == correct_label else -1

        # Move to the next email
        self.current_index += 1
        done = self.current_index >= len(self.emails)

        # Return step information
        return email_embedding, reward, done, {}

    def render(self, mode='human', close=False):
        # Optional: Implement if you want to visualize the environment
        pass

# Load the trained agent
agent = PPO.load('path_to_trained_model')  # Adjust the path to your trained model

# Load the test dataset
df_test = pd.read_csv('test_email_data.csv')  # Replace with your actual test data file

# Extract test embeddings and labels
test_embeddings = np.array(df_test['Email Embeddings'].apply(lambda x: np.fromstring(x[1:-1], sep=' ')))
test_labels = np.array(df_test['Type'])

# Create the test environment
test_env = EmailSecurityEnv(test_embeddings, test_labels)

# Initialize variables for predictions
predictions = []

# Run the agent on the test data
obs = test_env.reset()
done = False

while not done:
    action, _ = agent.predict(obs, deterministic=True)
    predictions.append(action)
    obs, _, done, _ = test_env.step(action)

# Map predictions to labels
labels = ["Spam" if pred == 1 else "Not Spam" for pred in predictions]

# Calculate performance metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Optionally, display some results
for text, label, prediction in zip(df_test['Email Text'], test_labels, predictions):
    print(f"Email: {text}\nTrue Label: {'Spam' if label == 1 else 'Not Spam'}, Prediction: {'Spam' if prediction == 1 else 'Not Spam'}\n")


# classify new email texts using the trained reinforcement learning agent

In [ ]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO  # Replace with your actual algorithm
from transformers import AutoTokenizer, AutoModel
import torch

# Define the custom environment for classification
class EmailClassificationEnv(gym.Env):
    def __init__(self, emails):
        super(EmailClassificationEnv, self).__init__()
        self.emails = emails
        self.current_index = 0
        self.action_space = spaces.Discrete(2)  # Actions: 0 - not spam, 1 - spam
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(emails.shape[1],), dtype=np.float32)

    def reset(self):
        self.current_index = 0
        return self.emails[self.current_index]

    def step(self, action):
        email_embedding = self.emails[self.current_index]
        self.current_index += 1
        done = self.current_index >= len(self.emails)
        return email_embedding, 0, done, {}

    def render(self, mode='human', close=False):
        pass

# Load the trained agent
agent = PPO.load('path_to_trained_model')  # Adjust the path to your trained model

# Initialize tokenizer and model for embedding generation
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Example new email texts
new_email_texts = [
    "Your account has been compromised, please click this link to secure it.",
    "Meeting tomorrow at 10 AM in the conference room.",
    "Congratulations! You've won a $1000 gift card. Click here to claim your prize."
]

# Generate embeddings for the new email texts
def generate_embeddings(email_texts):
    embeddings = []
    for text in email_texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

new_email_embeddings = generate_embeddings(new_email_texts)

# Create the environment with the new email embeddings
env = EmailClassificationEnv(new_email_embeddings)

# Classify the new emails
env.reset()
predictions = []

while True:
    action, _ = agent.predict(env.emails[env.current_index], deterministic=True)
    predictions.append(action)
    _, _, done, _ = env.step(action)
    if done:
        break

# Map predictions to labels
labels = ["Spam" if pred == 1 else "Not Spam" for pred in predictions]

# Print the results
for text, label in zip(new_email_texts, labels):
    print(f"Email: {text}\nClassification: {label}\n")
